# Segment Classifier

This notebook reports the methodology followed to build a machine learning-based classifier to automatically identify sections in job description documents. The model takes a sentence contained in a job description as input and produces as output the section that the sentence belongs to.

# Section 0. Preliminaries

## Load libraries

In [1]:
# update accordingly
run_on_google_colab = True
project_dir = '/content/drive/MyDrive'

if run_on_google_colab:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append(project_dir)

In [3]:
import json
import joblib
import numpy as np
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from text_processor import *
from utils import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Initialize constants

In [4]:
OVERWRITE_DOC2VEC = False
OVERWRITE_TRAINING = True
random_state = np.random.RandomState(1234)  # for reproducibility

## Load data

In [5]:
data_dir = f'{project_dir}/data'
data_fn = 'jobs_training.csv'
models_dir = f'{project_dir}/models'
output_dir = f'{project_dir}/outputs'

In [6]:
job_training_df = pd.read_csv(os.path.join(data_dir, data_fn))

## Preview data

Visualize the size of the data togethet with a small sample

In [7]:
print(f'dataset size: {job_training_df.shape[0]}x{job_training_df.shape[1]}')
print('-'*10)
job_training_df.head()

dataset size: 3885x4
----------


,job_id,segment_index,segment,section_label
0,05b865e93e8e46579075562865973d3b,0,Abbott is a global healthcare leader that help...,About Company
1,05b865e93e8e46579075562865973d3b,94,Our portfolio of life-changing technologies sp...,About Company
2,05b865e93e8e46579075562865973d3b,287,"Our 109,000 colleagues\nserve people in more t...",About Company
3,05b865e93e8e46579075562865973d3b,353,"**Tissue Trainer – St. Paul, MN**",Job Title
4,05b865e93e8e46579075562865973d3b,388,Our business purpose is to restore health and ...,About Company


Distribution of values by type of sections

In [8]:
job_training_df.section_label.value_counts()

Job Responsibilities/Summary    1453
Job Skills/Requirements         1012
Other                            506
About Company                    425
Benefits                         291
EOE/Diversity                    163
Job Title                         35
Name: section_label, dtype: int64

In [9]:
job_training_df.section_label.value_counts()/job_training_df.shape[0]

Job Responsibilities/Summary    0.374003
Job Skills/Requirements         0.260489
Other                           0.130245
About Company                   0.109395
Benefits                        0.074903
EOE/Diversity                   0.041956
Job Title                       0.009009
Name: section_label, dtype: float64

Previous output shows that there are `seven classes` classes, which are unbalaced. Now, check if there are missing values.

In [10]:
# check for null values
job_training_df.isnull().sum()

job_id           0
segment_index    0
segment          0
section_label    0
dtype: int64

Previous output unveils that there aren't missing values

In [11]:
job_training_df.groupby(['section_label'])['segment_index'].mean()

section_label
About Company                   1448.814118
Benefits                        2216.384880
EOE/Diversity                   4080.950920
Job Responsibilities/Summary    1501.912595
Job Skills/Requirements         2715.207510
Job Title                        296.085714
Other                           3379.217391
Name: segment_index, dtype: float64

From above we see that there seems to be relationship between segment index and sections. For example, `Job Title` segments have on average smaller segment indexes than `About Company`. Similary, segments in the `Other` section come on average by the end of the description but before `EOE/Diversity`.

---

# Section 1. Feature engineering

## Create syntactial features

Data are augmented with features related to the syntactical characteristics of job description sentences.

1. **Word count of sentences**: total number of words in sentences
2. **Character count of sentences**: total number of characters in sentences
3. **Average word density of sentences**: average length of words in sentences
4. **Upper case count in sentences**: total number of upper count words in sentences
5. **Title word count in sentences**: total number of title case words in sentences
6. **Noun count**: total number of nouns in sentences
7. **Verb count**: total number of verbs in sentences
8. **Adjective count**: total number of adjectives in sentences
9. **Pronouns count**: total number of pronouns in sentences

In [12]:
%%time
syn_features_df = pd.DataFrame()
for idx, row in job_training_df.iterrows():
  syn_features_df = pd.concat([syn_features_df, create_syntactical_features(row['segment'], idx)])

CPU times: user 34.8 s, sys: 355 ms, total: 35.2 s
Wall time: 35.6 s


In [13]:
syn_features_df.head()

,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,6,3,1,2,0,93,17,5.166667,0,1
1,10,2,4,0,0,192,24,7.680000,0,1
2,3,1,1,0,0,63,10,5.727273,0,1
3,9,0,1,0,0,33,6,4.714286,1,4
4,14,3,0,0,0,184,29,6.133333,2,1


Concate the new features with the job description data

In [14]:
job_training_df = pd.concat([job_training_df, syn_features_df.reindex(job_training_df.index)], axis=1)

In [15]:
job_training_df.head()

,job_id,segment_index,segment,section_label,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,05b865e93e8e46579075562865973d3b,0,Abbott is a global healthcare leader that help...,About Company,6,3,1,2,0,93,17,5.166667,0,1
1,05b865e93e8e46579075562865973d3b,94,Our portfolio of life-changing technologies sp...,About Company,10,2,4,0,0,192,24,7.680000,0,1
2,05b865e93e8e46579075562865973d3b,287,"Our 109,000 colleagues\nserve people in more t...",About Company,3,1,1,0,0,63,10,5.727273,0,1
3,05b865e93e8e46579075562865973d3b,353,"**Tissue Trainer – St. Paul, MN**",Job Title,9,0,1,0,0,33,6,4.714286,1,4
4,05b865e93e8e46579075562865973d3b,388,Our business purpose is to restore health and ...,About Company,14,3,0,0,0,184,29,6.133333,2,1


## Pre-process text

Pre-process job description sentences to be used  by the machine learning algorithms. In this sense, sentences are converted to a list of `lower case tokens`, `removing` in this process the `stops words`, `punctuations`, and `digits`.

In [16]:
processed_segs = preprocess_segments(job_training_df['segment'])

In [17]:
# Let's explore tokens of the first segment
processed_segs[0]

['abbott',
 'is',
 'a',
 'global',
 'healthcare',
 'leader',
 'that',
 'helps',
 'people',
 'live',
 'more',
 'fully',
 'at',
 'all',
 'stages',
 'of',
 'life']

In [18]:
# Check if processing tasks result in empty segments
idx_empty_segs = [idx for idx, seg in enumerate(processed_segs) if len(seg) == 0]
print(f'There are {len(idx_empty_segs)} empty segments')

There are 97 empty segments


In [19]:
# Let's look at how the empty segments look like
for idx in idx_empty_segs:
  print(f'[{idx}] {job_training_df.iloc[idx,2]}')

[171] ****
[175] ****
[178] ****
[505] **
[508] ****
[514] ****
[527] __
[535] :**
[552] 16.
[556] 19.
[558] 20.
[606] *
[657] *
[945] *
[951] *
[1101] *
[1105] *
[1191] •
[1199] •
[1203] •
[1354] *
[1365] *
[1413] :**
[1447] *
[1625] ·
[1627] ·
[1671] *
[1694] ·
[1697] ·
[1931] •
[1940] •
[1944] •
[1946] •
[2144] -
[2154] -
[2188] :**
[2685] ****
[2688] ****
[2690] ****
[2693] ****
[2696] ****
[2698] ****
[2818] *
[2873] ...
[2987] *
[2989] *
[3002] *
[3004] *
[3025] •
[3028] •
[3030] •
[3033] •
[3038] •
[3040] •
[3042] •
[3044] •
[3046] •
[3048] •
[3051] •
[3053] •
[3055] •
[3057] •
[3059] •
[3061] •
[3066] •
[3070] •
[3182] ****
[3249] _**
[3397] ?
[3423] *
[3428] *
[3433] *
[3435] *
[3440] *
[3442] *
[3447] *
[3451] *
[3453] *
[3455] *
[3457] *
[3461] *
[3488] *
[3492] *
[3600] *
[3603] *
[3693] -
[3694] 160199
[3699] ®
[3703] ®
[3704] .
[3712] :
[3739] 5
[3756] :
[3759] :
[3762] :
[3765] :
[3850] •


In [20]:
# Get rid empty segments
processed_segs = [seg for seg in processed_segs if len(seg) > 0]
print(f'In total {len(processed_segs)} segments will be used')
# Get rid of rows with empty segments
processed_job_training_df = job_training_df.drop(idx_empty_segs).reset_index(drop=True)
assert len(processed_segs) == processed_job_training_df.shape[0], 'Number of rows in dataframe should be equal to the processed segments'

In total 3788 segments will be used


## Encode labels

The target variable `section_label` contains categorical data, which need to be converted to numbers before usign them to train machine learning algorithms.

In [21]:
labels = list(processed_job_training_df['section_label'].unique())
label_values = list(processed_job_training_df['section_label'].values)
encoded_labels = encode_labels(labels, label_values, models_dir)
assert len(encoded_labels) == len(processed_segs), 'Number of lables should be equal to the processed segments'

## Split dataset

Split the dataset into train and test, holding 20% for testing

In [22]:
features, labels = pd.concat([pd.Series(processed_segs, name='segment_tokens'), processed_job_training_df['segment_index'],
                              processed_job_training_df.iloc[:,4:]], axis=1), encoded_labels
features_train, features_test, y_train, y_test = train_test_split(features, labels, random_state=random_state,
                                                                  test_size=0.20, stratify=labels)

In [23]:
features_train.head(3)

,segment_tokens,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
3206,"[is, involved, with, the, production, and, coo...",576,3,2,1,1,0,82,11,6.833333,0,1
2203,"[assess, liability, and, damages, and, develop...",738,9,1,1,0,0,132,19,6.600000,0,2
427,"[key, responsibilities]",466,1,0,1,0,0,21,2,7.000000,0,2


In [24]:
y_train[:3]

array([3, 3, 3])

Check if the proportion of target classes has been preserved

In [25]:
labels_train = decode_labels(y_train, os.path.join(models_dir, 'encoder_classes.npy'))
labels_train[:3]

['Job Responsibilities/Summary',
 'Job Responsibilities/Summary',
 'Job Responsibilities/Summary']

In [26]:
unique, counts = np.unique(labels_train, return_counts=True)
dict(zip(unique, counts/len(labels_train)))

{'About Company': 0.11089108910891089,
 'Benefits': 0.07491749174917492,
 'EOE/Diversity': 0.0429042904290429,
 'Job Responsibilities/Summary': 0.3693069306930693,
 'Job Skills/Requirements': 0.262046204620462,
 'Job Title': 0.009240924092409241,
 'Other': 0.1306930693069307}

## Scale non-textual features

Scale numerical features (i.e., the syntactical created before and the `segment_index`) so values are closer to word vectors that are generated in the next steps

In [27]:
scaled_features_array = scale_features(np.array(features_train.iloc[:,1:]), models_dir)
scaled_features_df = pd.DataFrame(scaled_features_array, columns=list(features_train.columns)[1:])

In [28]:
scaled_features_df.head()

,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,-0.840045,-0.549759,-0.136614,-0.385304,0.972226,-0.176252,-0.292575,-0.364450,0.415860,-0.199011,-0.433468
1,-0.757201,0.124809,-0.531328,-0.385304,-0.482161,-0.176252,0.104279,0.088912,0.275486,-0.199011,-0.160373
2,-0.896297,-0.774615,-0.926041,-0.385304,-0.482161,-0.176252,-0.776737,-0.874483,0.516127,-0.199011,-0.160373
3,-0.887092,-0.549759,0.258099,-0.385304,2.426613,-0.176252,0.072530,-0.024429,0.582972,-0.199011,-0.433468
4,-0.894252,1.136662,-0.531328,0.484024,-0.482161,-0.176252,0.826553,0.882296,0.250714,-0.199011,1.205101


## Create embeddings using doc2vec

Job descriptions are used to create word embeddings of sentences. `Doc2vec` is employed to learn the vector the representation of job description sentences. The vector length and learning approach were decided based on the results of experiments with different combinations of sizes (`50`, `100`, `200`, `300`, `400`, `500`) and the doc2vec approaches `distributed memory` and `distributed bag of words` implemented by the [Gensim](https://radimrehurek.com/gensim/models/doc2vec.html) library.

In [29]:
%%time
doc2vec_model_fn = os.path.join(models_dir, 'doc2vec.model')
if OVERWRITE_DOC2VEC or not os.path.isfile(doc2vec_model_fn):
  hyper_params = {
      'vector_size': 500,
      'alpha': 0.025,
      'min_count': 5,
      'dm': 0,
      'epochs': 100
  }
  print('Preparing sentences for training...')
  train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(list(features_train['segment_tokens']))]
  print('-'*10)
  print('Training embeddings...')
  doc2vec_model = Doc2Vec(**hyper_params)
  doc2vec_model.build_vocab(train_doc2vec)
  doc2vec_model.train(train_doc2vec, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
  print('-'*10)
  print('Doc2vec model successfully created!')
  print('-'*10)
  print('Saving model...')
  print('-'*10)
  doc2vec_model.save(doc2vec_model_fn)
else:
  print('Loading doc2vec model...')
  doc2vec_model = Doc2Vec.load(doc2vec_model_fn)

Loading doc2vec model...
CPU times: user 25.8 ms, sys: 17.1 ms, total: 42.9 ms
Wall time: 617 ms


## Create doc vector features

Pre-trained embeddings are used to generate vectors for sentences

In [30]:
%%time
print('Creating vectors...')
if doc2vec_model:
  train_vectors = [doc2vec_model.infer_vector(seg_tokens) for seg_tokens in list(features_train['segment_tokens'])]
print('-'*10)

Creating vectors...
----------
CPU times: user 13.2 s, sys: 40.8 ms, total: 13.2 s
Wall time: 13.4 s


In [31]:
# let's explore an example of a generated doc vector
train_vectors[0]

array([-2.82705091e-02, -2.76618838e-01,  8.07070658e-02,  3.32186744e-02,
        3.24100137e-01,  1.05206840e-01, -3.85255188e-01,  3.02765369e-02,
        3.11930757e-02, -3.14988852e-01, -2.79536583e-02, -1.56732693e-01,
       -7.20462501e-02, -1.32282808e-01,  1.23934120e-01,  3.61815318e-02,
        6.96084499e-02,  3.43142934e-02,  2.13810161e-01,  1.72980085e-01,
       -4.95111793e-02,  1.16265364e-01,  2.47769728e-01,  9.43451449e-02,
        1.24633946e-01, -1.39879450e-01, -3.58885795e-01,  1.25981674e-01,
       -2.05853686e-01,  2.45698422e-01,  2.45287925e-01,  5.80426939e-02,
        1.87979192e-01,  1.35785267e-01, -1.05113655e-01, -8.28894079e-02,
       -1.23204269e-01, -3.28538828e-02, -2.97754109e-01, -3.92719269e-01,
        2.27378324e-01,  9.84204561e-02,  5.81392497e-02, -6.15320355e-02,
        1.95460811e-01, -7.14540705e-02, -4.20267880e-02, -1.98003829e-01,
       -9.58323926e-02, -6.37337118e-02, -5.05187847e-02, -1.36601672e-01,
        2.56863311e-02,  

---

# Section 2. Model building

## Build ML model

Here the machine learning classifier is built. The decision on the algorithms to be used was based on algorithms that have been reported to perform well on unbalanced, small, and textual datasets (e.g., [Text Classification with Extremely Small Datasets](https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2)). Therefore, we try the following algorithms: `Support Vector Machine` and `Logistic Regression`.

In bulding the model we proceed as following:

* Grid search is used to find the optimal combination of hyperparameters;
* Best models for each algortihm are pre-selected for training;
* Testing performance of best models are compared and the model that shows the best results is selected.

Models are trained using the cross-validation approach. The performance metric to be optimized is weighted `F1` because it provides an adequate compromise between acceptable coverage and the correct identification of segments.

### Train classifiers

In [32]:
# concatenate word vectors with numerical features
X_train = np.concatenate((train_vectors, scaled_features_df), axis=1)

In [33]:
def get_lr_parameters():
    """
    Define hyper-parameters of
    Logistic Regression.

    Inspired by
    https://notebook.community/tpin3694/tpin3694.github.io/machine-learning/.ipynb_checkpoints/hyperparameter_tuning_using_grid_search-checkpoint
    """
    param_grid = {
        'penalty': ['l2', None],
        'C': np.logspace(0, 4, 10)
    }
    return param_grid

In [34]:
def get_svm_parameters():
    """
    Define hyper-parameters of
    Support Vector Machine.

    Inspired by
    https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv
    """

    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001, 'scale'],
        'kernel': ['linear', 'rbf', 'poly']
    }
    return param_grid

In [35]:
def filter_params(model_params, hyperparams):
  filtered_params = {}
  for param, value in model_params.items():
    if param in hyperparams:
      filtered_params[param] = value
  return filtered_params

#### Hyper-parameters optimization

In [36]:
%%time
output_training_file_path = os.path.join(output_dir, 'output_ml_training.json')
if not os.path.isfile(output_training_file_path) or OVERWRITE_TRAINING:
  # discard 20% of the training sample
  X_hpo_train, _, y_hpo_train, _ = train_test_split(
      X_train, y_train, random_state=random_state,
      test_size=0.20, stratify=y_train
  )
  metric = {
      'name': 'f1',
      'obj': make_scorer(f1_score, average='weighted')
  }
  training_outputs = []
  for algorithm in ['LR', 'SVM']:
    print(f'Training {algorithm} models with different combination of hyperparameters...')
    if algorithm == 'SVM':
      # class_weight='balanced' is used to indicate that classes are not balanced in the dataset
      classifier = SVC(random_state=random_state, class_weight='balanced')
      hyperparams = get_svm_parameters()
    else:
      classifier = LogisticRegression(random_state=random_state,
                                      class_weight='balanced', max_iter=200,
                                      solver='lbfgs')
      hyperparams = get_lr_parameters()
    # create grid search
    clf = GridSearchCV(classifier, param_grid=hyperparams, scoring=metric['obj'],
                       verbose=2, error_score='raise', n_jobs=-1)
    # do grid search
    best_model = clf.fit(X_hpo_train, y_hpo_train)
    score = best_model.best_score_
    print(f'Finished training {algorithm} models, {metric["name"]} score of best model: {score}')
    print(f'Hyperparameters of the best {algorithm} model are:')
    print(filter_params(best_model.best_estimator_.get_params(), hyperparams))
    print(f'Saving the best model {algorithm} model...')
    model_name = f'{algorithm.lower()}_model.joblib'
    model_file_path = os.path.join(models_dir, model_name)
    model_dict = dict(model=best_model)
    joblib.dump(model_dict, model_file_path)
    training_outputs.append(
      {
          'algorithm': algorithm,
          'hyperparams': filter_params(best_model.best_estimator_.get_params(), hyperparams),
          'metric': metric['name'],
          'score': score,
          'model_file_path': model_file_path
      }
    )
    print('-'*10)
  # save training outputs
  print('Saving outputs...')
  with open(output_training_file_path, 'w') as f:
      json.dump(training_outputs, f, indent=4)
  print('-'*10)
else:
  print('Skipping, training outputs already exist...')

Training LR models with different combination of hyperparameters...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Finished training LR models, f1 score of best model: 0.6838951608498576
Hyperparameters of the best LR model are:
{'C': 1.0, 'penalty': 'l2'}
Saving the best model LR model...
----------
Training SVM models with different combination of hyperparameters...
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Finished training SVM models, f1 score of best model: 0.7243679114796706
Hyperparameters of the best SVM model are:
{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}
Saving the best model SVM model...
----------
Saving outputs...
----------
CPU times: user 14.5 s, sys: 2.36 s, total: 16.8 s
Wall time: 13min 4s


#### Train best clasifiers

Linear regression

In [37]:
with open(output_training_file_path, 'r') as f:
    training_outputs = json.load(f)
# look for best hyper-parameters combination
for training in training_outputs:
  if training['algorithm'] == 'LR':
    params = training['hyperparams']
# define LR classifier
lr_classifier = LogisticRegression(
    random_state=random_state,
    class_weight='balanced',
    max_iter=200,
    solver='lbfgs',
    **params
)

In [38]:
%%time
lr_classifier.fit(X_train, y_train)

CPU times: user 3.56 s, sys: 1.24 s, total: 4.8 s
Wall time: 4.2 s


LogisticRegression(class_weight='balanced', max_iter=200,
                   random_state=RandomState(MT19937) at 0x7F4BD6CEFE40)

#### Support vector machine

In [39]:
with open(output_training_file_path, 'r') as f:
    training_outputs = json.load(f)
# look for best hyper-parameters combination
for training in training_outputs:
  if training['algorithm'] == 'SVM':
    params = training['hyperparams']
# define SVM classifier
svm_classifier = SVC(
    random_state=random_state,
    class_weight='balanced',
    **params
)

In [40]:
%%time
svm_classifier.fit(X_train, y_train)

CPU times: user 2.49 s, sys: 39.7 ms, total: 2.53 s
Wall time: 2.58 s


SVC(C=10, class_weight='balanced', kernel='poly',
    random_state=RandomState(MT19937) at 0x7F4BD6CEFE40)

### Evaluate best classifiers

#### Prepare test set

Scale numerical features

In [41]:
scaled_features_array = scale_features(np.array(features_test.iloc[:,1:]), models_dir)
scaled_features_df = pd.DataFrame(scaled_features_array, columns=list(features_train.columns)[1:])

Generate doc vectors of segments in the test split

In [42]:
# load doc2vec model
doc2vec_model_file_path = os.path.join(models_dir, 'doc2vec.model')
doc2vec_model = Doc2Vec.load(doc2vec_model_file_path)
test_vectors =  [doc2vec_model.infer_vector(seg_tokens) for seg_tokens in list(features_test.segment_tokens)]

Concatenate vectors with numerical features

In [43]:
X_test = np.concatenate((test_vectors, scaled_features_df), axis=1)

#### **Model**: Logistic Regression classifier

Make predictions

In [44]:
preds = lr_classifier.predict(X_test)

Evaluate model performance

In [45]:
enconder_file_path = os.path.join(models_dir, 'encoder_classes.npy')
class_nums = list(range(0,7))
print(classification_report(y_test, preds, target_names=decode_labels(class_nums, enconder_file_path)))

                              precision    recall  f1-score   support

               About Company       0.70      0.67      0.68        84
                    Benefits       0.56      0.77      0.65        57
               EOE/Diversity       0.76      0.85      0.80        33
Job Responsibilities/Summary       0.82      0.66      0.73       280
     Job Skills/Requirements       0.78      0.80      0.79       198
                   Job Title       0.11      0.29      0.15         7
                       Other       0.62      0.71      0.66        99

                    accuracy                           0.72       758
                   macro avg       0.62      0.68      0.64       758
                weighted avg       0.74      0.72      0.72       758



#### **Model**: Support Vector Machine classifier

Make predictions

In [46]:
preds = svm_classifier.predict(X_test)

Evaluate model performance

In [47]:
enconder_file_path = os.path.join(models_dir, 'encoder_classes.npy')
class_nums = list(range(0,7))
print(classification_report(y_test, preds, target_names=decode_labels(class_nums, enconder_file_path)))

                              precision    recall  f1-score   support

               About Company       0.67      0.65      0.66        84
                    Benefits       0.73      0.65      0.69        57
               EOE/Diversity       0.93      0.79      0.85        33
Job Responsibilities/Summary       0.80      0.78      0.79       280
     Job Skills/Requirements       0.77      0.79      0.78       198
                   Job Title       0.40      0.29      0.33         7
                       Other       0.70      0.80      0.75        99

                    accuracy                           0.76       758
                   macro avg       0.71      0.68      0.69       758
                weighted avg       0.76      0.76      0.76       758



### Select best-perfomed model

> From results above it is clear that the **`support vector machine`** model shows better a better performance with an **`accuracy`** of **`0.76`** and a similar weighted average **`f1`** in comparison with its **`logistic regression`** counterpart, which shows an **`accuracy`** of **`0.72`** and a weighted average **`f1`** of **`0.72`**. Therefore, the **`support vector machine`** model will be used for predictions.

---

# Section 3. Use case

### Predict section of segments

Solution is checked by predicting the section of a given sentence (and sentence index) taken from the dataset `jobs_test`.

In [48]:
segment = 'The company began more than 100 years ago in Tulsa and has successfully diversified into a variety of industries, businesses and geographies. .'
segment_idx = 341

In [49]:
segments_df = pd.DataFrame({'segment': [segment], 'segment_index': [segment_idx]})
segments_df.head()

,segment,segment_index
0,The company began more than 100 years ago in T...,341


Create syntactical features for the segment

In [50]:
num_features_df = create_syntactical_features(segment)
num_features_df.head()

,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,7,3,1,2,0,143,23,5.958333,0,2


In [51]:
num_features_df = pd.concat([segments_df['segment_index'], num_features_df], axis=1)
num_features_df.head()

,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,341,7,3,1,2,0,143,23,5.958333,0,2


Scale numerical features

In [52]:
scaled_features_array = scale_features(np.array(num_features_df.iloc[:,:]), models_dir)
scaled_num_features_df = pd.DataFrame(scaled_features_array, columns=list(features_train.columns)[1:])
scaled_num_features_df.head()

,segment_index,noun_count,verb_count,adj_count,adv_count,pron_count,chart_count,word_count,word_density,upper_case_word_count,title_word_count
0,-0.96022,-0.100047,0.258099,-0.385304,2.426613,-0.176252,0.191587,0.315593,-0.110543,-0.199011,-0.160373


Pre-process segment

In [53]:
processed_segs = preprocess_segments(segments_df['segment'])
processed_segs[0]

['the',
 'company',
 'began',
 'more',
 'than',
 'years',
 'ago',
 'in',
 'tulsa',
 'and',
 'has',
 'successfully',
 'diversified',
 'into',
 'a',
 'variety',
 'of',
 'industries',
 'businesses',
 'and',
 'geographies']

Generate doc vector for segment

In [54]:
# load doc2vec model
doc2vec_model_file_path = os.path.join(models_dir, 'doc2vec.model')
doc2vec_model = Doc2Vec.load(doc2vec_model_file_path)
segment_vector =  [doc2vec_model.infer_vector(seg_tokens) for seg_tokens in processed_segs]

Concatenate vector with numerical features

In [55]:
seg_features = np.concatenate((segment_vector, scaled_num_features_df), axis=1)

Make prediction

In [56]:
pred = svm_classifier.predict(seg_features)

Output prediction result

In [57]:
class_names = decode_labels(list(range(0,7)), os.path.join(models_dir, 'encoder_classes.npy'))
for class_num, class_name in zip(list(range(0,7)), class_names):
  if class_num == pred[0]:
    print('Prediction result')
    print('-'*10)
    print(f'Segment: {segment}')
    print(f'Predicted section: {class_name}')
    print('-'*10)
    break

Prediction result
----------
Segment: The company began more than 100 years ago in Tulsa and has successfully diversified into a variety of industries, businesses and geographies. .
Predicted section: About Company
----------


---